# Edge-Vertex Continuous Collision Detection

Given a point that moves over time, $\vec{p}_0(t)$, and an edge, $\vec{e}(s, t)$, comprised of two points, $\vec{p}_1(t)$ and $\vec{p}_2(t)$, we want to find the time of impact between the point and edge. Let us assume a normalized time step $t=0$ to $t=1$. We known the point positions for $t=0$ and $t=1$. Let us also assume that for all points the trajectory is linear and the velocity is constant for all points. We can write the trajectories as 

\begin{align}
    \vec{p}_i(t) &= [\vec{p}_i(1) - \vec{p}_i(0)] t + \vec{p}_i(0) & \forall i \in \{0, 1, 2\}, t \in [0, 1]\\
    e(s, t) &= (\vec{p}_2(t) - \vec{p}_1(t)) s + \vec{p}_1(t) & \forall s \in [0, 1] 
\end{align}

In [20]:
# Compute a polynomial for the intersection of a point and edge both moving
# through time.
# General idea: http://www.sci.utah.edu/~kpotter/publications/ramsey-2004-RBPI.pdf

import numpy
import sympy
from IPython.display import display, Markdown, Math, Latex
from sympy.printing.latex import latex
# sympy.init_printing(use_latex='mathjax')

# s is the spatial parameterization for the edge
# t is the temporal parameterization
s, t = sympy.symbols("s t")

def display_eq(name, value):
    display(Math("%s:=%s" % (name, latex(value))))

In [11]:
dimensions = 2  # Number of dimensions of the points and edge
# For each dimenstion write the trajectory of the point and edge
d = "y" if dimensions == 2 else "z"
# Three points, one free and two for the edge, each with a velocity
p = numpy.reshape(sympy.symbols(f"p_0:3_x:{d}", real=True), (-1, dimensions))
v = numpy.reshape(sympy.symbols(f"v_0:3_x:{d}", real=True), (-1, dimensions))

In [12]:
# Write the point's trajectory as a line in time
pt = p + t * v
# Write the dimensions of the edge
e = (pt[2,:] - pt[1,:]) * s + pt[1,:]

In [58]:
sn = [None] * dimensions
sd = [None] * dimensions
seq = [None] * dimensions
for i in range(0, dimensions):
    # Equate the point and edge in the current dimension
    eq = sympy.Eq(pt[0,i], e[i])
    # Solve for the spatial parameterization along the edge
    s_ = sympy.solve(eq, s)[0]
    display_eq("s_%i" % i, s_)
    # Split the spatial parameterization into a numerator and denominator
    # NB: The denominator can be zero if the edge is degenerate
    sn[i], sd[i] = sympy.fraction(s_)

# Equate the different dimensions of the spatial parameterization
if dimensions == 2:
    eq1 = sympy.Eq(sn[0] * sd[1], sn[1] * sd[0])
elif dimensions == 3:
    eq1 = sympy.Eq(sd[2] * (sn[0] * sd[1] - sn[1] * sd[0]), sn[0])
else:
    raise Exception("Invalid dimension option")

# Write the equations as a polynomial in t
t_poly = sympy.polys.polytools.poly_from_expr(eq1, t)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [25]:
coeffs = t_poly[0].all_coeffs()
coeffs_names = ['a','b','c']
for i, coeff in enumerate(coeffs):
    display_eq(coeffs_names[i], sympy.factor(coeff))

<IPython.core.display.Latex object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Equation for $t$ in 2D:
We want to solve the quadratic equation $a t^2 + b t + c = 0$ with the coefficient defined above.

Using the formula $$t = \frac{-b \pm \sqrt{(b^2 - 4 a c)}}{2a}$$ is unstable for values of $a$ near zero.

Another alternative is the **Citardauq Formula** $$t = \frac{-2c}{ |b| \pm \sqrt{(b^2 - 4 a c)}}$$
where one of the roots is stable when $a \rightarrow 0$. The other root goes to infinitity (as $-b + |b|$) goes to zero.

What we will do is use one or the other one depending on the values of $a$ and $b$.

In [55]:
from math import sqrt
def solve_quadratic_v1(a, b, c):
    D = b**2 - 4*a*c
    x1, x2 = None, None
    if D > 0 :
        if a != 0 :
            x1 = (- b + sqrt(D))/(2*a)
            x2 = (- b - sqrt(D))/(2*a)
        elif b != 0:
            x1 = - c /b
    return x1, x2

def solve_quadratic_v2(a, b, c):
    D = b**2 - 4*a*c
    x1, x2 = None, None
    if D > 0:
        if b > 0 :
            x1 = -2*c/(b + sqrt(D))
            if a != 0 :
                x2 = (-b - sqrt(D))/(2*a)
        else:
            x2 = -2*c/(-b + sqrt(D))
            if a != 0:
                x1 = (-b + sqrt(D))/(2*a)
    return x1, x2

In [57]:
x = solve_quadratic_v1(2.0, 1.0, -1.0)
y = solve_quadratic_v2(2.0, 1.0, -1.0)
print(x, y)

(0.5, -1.0) (0.5, -1.0)
